# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a classification problem, since we are classifying students into two groups: those that need intervention, and those that don't. A regression problem would be something like quantifying the probability of failure, for example.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [3]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [6]:
# Calculate number of students
n_students = (student_data.shape)[0] 

# Calculate number of features. We need to subtract 1 from the number of columns because
# one of the columns is the target column
n_features = (student_data.shape)[1] - 1 

# Calculate passing students
n_passed = sum(student_data.get("passed") == "yes")

# Calculate failing students
n_failed = n_students - n_passed

# Calculate graduation rate
grad_rate = float(n_passed) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [7]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [8]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [9]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# Get test fraction
testsize = float(num_test) / (num_train + num_test)

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=testsize, random_state=1)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

**Answer: **

This is a supervised learning problem since we have all of the labels for the training data. Moreover, there are many features, which means that I need algorithms that can deal with many features for fewer training examples. SVM would be a good fit for it, and also decision trees. I will also try neural networks, because I am least comfortable with NNs and want to spend some time implementing NNs.

The general applications, strengths and weakness of each model:

##### Decision Trees

. ***General Applications*** 

Decision trees are used extensively in multi-class classification problems the features are discrete. In the case of continuous features, bins may be used. Some actual applications are

1. [Filtering data from the Hubble Space Telescope](https://www.jstor.org/stable/40680533?seq=1#page_scan_tab_contents)
2. Text processing
3. Data Mining

. ***Strengths***

Some of the strengths of decision trees are:

1. Easy to understand for humans
2. Gives reasonable answers with limited training data
3. Easy to combine with other algorithms, especially boosting
4. Very short training and testing time relative to optimization algorithms such as SVM, LR

. ***Weaknesses***

1. Prone to overfitting
2. Small changes in training data can cause significant changes in the tree. The trees are affected strongly by outliers and the algorithm designers have to be careful

. *** Why I chose decision trees ***

Given that the dataset has fewer examples and has many features, I felt that decision trees would be a good algorithm to try on this data set. Also, it makes intuitive sense to ask for a student: what is their family size? are they in a relationship?

##### K-nearest neighbors

. ***General Applications*** 

The KNN algorithm is used in a variety of applications, including:

1. Facial recognition 
2. Image search
3. [Protein protein interaction. Huh?!](https://saravananthirumuruganathan.wordpress.com/2010/05/17/a-detailed-introduction-to-k-nearest-neighbor-knn-algorithm/)

. ***Strengths***

1. Very fast training time, since the training is basically zero. Given the training data, you can directly head to classification
2. There is no need to know anything about the training data apart from the definition of a distance function
3. Complex, nonlinear decision boundaries are handled by the nature of the classification process

. ***Weaknesses*** 

1. No intuition of the physical model is provided by the algorithm
2. The complexity KNN algorithm grows very quickly as the training set increases in size, since every training point has to be considered for classification of a new point
3. The curse of dimensionality is strong in this one

. *** Why I chose this algorithm ***

Given a student, it makes for a reasonable hypothesis that his performance would match that of other students, provided we have chosen the right metrics. As the quote goes, ["You are the average of the five people who you spend the most time with."](http://www.goodreads.com/quotes/1798-you-are-the-average-of-the-five-people-you-spend) . This is why I felt that KNN would be a good algorithm to try here. 

##### SVM

. ***General Applications*** 

SVMs are used in many, many ML applications are generally the first stop in the selection of an algorithm. For example, they are used in:
1. Image recognition
2. Search engines 
3. Databases
4. Extension to Structured SVM allows for usage with NLP as well

. ***Strengths***

1. Easily generalizes to high dimensional spaces
2. The curse of dimensionality is sort-of weakened by the algorithm only having to consider the support vectors, which in the usual case, provides a sparse matrix for classification
3. Using kernel functions, a linear algorithm can be magically applied to nonlinear classification problems

. ***Weaknesses***

1. Not very strong when there are many features relative to the sample size. In this case, decision trees might be a better idea
2. Not easy to implement multiclass classification
3. Kernel parameters and the choice of kernel is kind of a black box
4. Slow training time, since there is a quadratic optimization going on every time
5. Not easy to get a probabilistic score for each training example, which would have been useful to the board of directors at the school in this problem
    
. *** Why I chose this algorithm ***

The number of features here is quite high, and hence the feature space is high-dimensional, which makes SVM a good fit for this problem. However, the real reason I chose it here is that I feel that SVM is always a good benchmark to start playing with the features and the data. I expected that SVM would do worse than the other two, but, as it turned out I was wrong!

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [10]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [12]:
# Import the three supervised learning models from sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Initialize the three models
clf_DTC = DecisionTreeClassifier(random_state=0)
clf_SVC = SVC()
clf_KNC = KNeighborsClassifier()

# Set up the training set sizes
X_train_100 = X_train[1:100]
y_train_100 = y_train[1:100]

X_train_200 = X_train[1:200]
y_train_200 = y_train[1:200]

X_train_300 = X_train
y_train_300 = y_train

# Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_DTC, clf_SVC, clf_KNC]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0053 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6441.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0016 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7077.
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0045 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6720.
Training a SVC using a training set size of 100. . .
Trained model in 0.0043 seconds
Made predictions in 0.0016 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.8333.
Training a SVC using a training set size of 200. . 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Decision Trees**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0015           |         0.0004         |     1.0000       |     0.7111      |
| 200               |        0.0024           |         0.0005         |     1.0000       |     0.7481      |
| 300               |        0.0041           |         0.0004         |     1.0000       |     0.6720      |

** Classifer 2 - K Nearest Neighbor**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0014           |         0.0018         |      0.7941      |      0.7727     |
| 200               |        0.0014           |         0.0044         |      0.8211      |      0.7971     |
| 300               |        0.0017           |         0.0080         |      0.8588      |      0.7681     |

** Classifer 3 - SVM**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0057            |           0.0012       |      0.8649      |      0.8442     |
| 200               |       0.0047            |           0.0022       |      0.8608      |         0.8354  |
| 300               |       0.0073            |           0.0050       |      0.8584      |     0.8462      |



## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Dear Board of Supervisors,

I present to you my algorithm of choice, the Support Vector Machine! After extensive testing for half an hour, I have found that the SVM outperforms all other algorithms. 

It takes longest to train the algorithm of the algorithms that I chose. However, given that your school has only hundreds of students in a batch, I believe this is a fair compromise. If you would rather have a blazing fast algorithm at a slight performance hit, let me know and I will change the algorithm.

When I started this investigation, I believed that I would find Decision Trees most preferbale, since the data you provided me has many features for a relatively small number of students. The DT algorithm is, indeed, the best when you consider testing time. But, I believe the tradeoff in accuracy is not worth it, given the number of students you have. 

Sincerely,
Pritam Sukumar

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

**Answer: **

Dear Board of Supervisors,

The SVM is an algorithm commonly used to separate (classify) data. In this case, for example, given a new student's data, we want to know whether he or she is at risk of failing to graduate. Like most Machine Learning algorithm, the algorithm works in two stages: training and testing. I will describe each of these stages separately to give you a better idea.

*** Training *** 
we use for training, the data for students for whom we already know the result. Given sufficient data of both sides (passing and not-passing), the algorithm lands upon a boundary that minimizes the error of misclassification. That means that most of the passing students will be on one side of this boundary, and most of the failing students will be on the other side.

Imagine data on a plane, and having only two features (along each of the axes). If we've chosen our features wisely and the data can be separated with a line, the SVM algorithm will find a line that separates the data into two regions. In the figure below ([ref](https://www.mql5.com/en/articles/584)), you can see the SVM algorithm converging step by step on the boundary. 

![alt text](svmregression.gif "Title")

What is interesting in the above is that the data is **not** linearly separable. How then, you might ask, did the SVM come up with this separator? What we can do with SVM is that we can introduce an extra dimension. In this case, that would mean adding a value corresponding to each of the points on the plane. So in this third dimension, each point can corresponds to a point sticking out of the page at different heights. What we can do is choose a mapping of "point" to "height" so that we can be sure that the data can in fact be separated when this extra dimension is introduced. Isn't that cool? This is what is called, in MLspeak, the Kernel Trick. And the mapping is called a Kernel.

The algorithm now goes further and says that the only points that are of interest are the support vectors, which are the points on both sides closest to the boundary. They, in effect, support this boundary. The advantage of these points is that, in classifying a new point, the algorithm only needs to consider the values at these support vectors, rather than the whole set of training data. 

There are many parameters that can be tuned for SVMs, including the kernel type, the learning parameter (which affects how much the line moves in each step), and so on, which makes SVMs extremely hackable and good to play with, for various problems.

*** Prediction *** 
At the end of training, we get a simple equation that we just input the features for a new student, and then we get the output! 

Cool, eh?

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [14]:
from sklearn import svm, grid_search
from sklearn.metrics import f1_score, make_scorer
from sklearn.svm import SVC

parameters = {'kernel':('rbf', 'linear', 'poly'), 'C':range(1, 4)}

# TODO: Initialize the classifier
svr = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
clf_grid = grid_search.GridSearchCV(svr, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = clf_grid.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0058 seconds.
Tuned model has a training F1 score of 0.8584.
Made predictions in 0.0017 seconds.
Tuned model has a testing F1 score of 0.8462.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final F1 score for training and testing are 0.8584 and 0.8462 respectively. This is exactly the same as the untuned model, which tells us the SVM did pretty well just by default!

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.